In [564]:
import pandas as pd
import csv
import os
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import RFECV
import joblib
from config import user_name,password

In [442]:
btc = pd.read_csv("data/BTC-USD.csv")
btc = btc.drop(columns=["Adj Close","Open","High","Low","Volume"])
btc['Date']= pd.to_datetime(btc['Date'])
btc = btc.dropna()
btc_data = btc.iloc[::-1]
# btc_data.head()

In [443]:
btc_data = btc_data.rename(columns={'Close': 'Close_Btc'})
btc_data['Date'] = pd.to_datetime(btc_data['Date'])
btc_data['Day_Btc'] = btc_data['Date'].dt.day_name()
btc_data = btc_data.sort_values(by="Date")
# btc_data

In [444]:
btc_data_clean = btc_data.loc[((btc_data['Day_Btc'] != "Saturday") & (btc_data['Day_Btc'] != 'Sunday'))]
# btc_data_clean

In [575]:
# btc_data_clean = btc_data_clean.drop(columns=["Day_Btc"])
btc_data_clean["BTC_Change%"] = btc_data_clean["Close_Btc"].pct_change()
btc_data_clean['BTC_Change%_lag'] = btc_data_clean["BTC_Change%"].shift(1)

# Construction direction of BITCOIN movement
btc_data_clean['BTC_dir'] = (btc_data_clean["BTC_Change%"]>0).astype(float)
btc_data_clean['ETH_dir_lag'] = btc_data_clean['BTC_dir'].shift(1)
# btc_data_clean

In [446]:
btc_data_clean.dtypes

Date           datetime64[ns]
Close_Btc             float64
BTC_Change%           float64
dtype: object

In [447]:
eth = pd.read_csv("data/ETH-USD.csv")
eth = eth.drop(columns=["Adj Close","Open","High","Low","Volume"])
eth['Date']= pd.to_datetime(eth['Date'])
eth_data = eth.dropna()
eth_data = eth_data.iloc[::-1]
# eth_data.head()

In [448]:
eth_data = eth_data.rename(columns={'Close': 'Close_Eth'})
eth_data['Date'] = pd.to_datetime(eth_data['Date'])
eth_data['Day_Eth'] = eth_data['Date'].dt.day_name()
eth_data = eth_data.sort_values(by="Date")
# eth_data

In [449]:
eth_data_clean = eth_data.loc[((eth_data['Day_Eth'] != "Saturday") & (eth_data['Day_Eth'] != 'Sunday'))]
# eth_data_clean

In [521]:
# eth_data_clean = eth_data_clean.drop(columns=["Day_Eth"])
eth_data_clean["ETH_Change%"] = eth_data_clean["Close_Eth"].pct_change()
eth_data_clean['ETH_Change%_lag'] = eth_data_clean["ETH_Change%"].shift(1)

# Construction direction of Ethereum movement
eth_data_clean['ETH_dir'] = (eth_data_clean["ETH_Change%"]>0).astype(float)
eth_data_clean['ETH_dir_lag'] = eth_data_clean['ETH_dir'].shift(1)
eth_data_clean.head()

,Date,Close_Eth,ETH_Change%,ETH_Change%_lag,ETH_dir,ETH_dir_lag
2,2016-10-10,11.7555,NaN,NaN,0.0,NaN
3,2016-10-11,11.7857,0.002569,NaN,1.0,0.0
4,2016-10-12,11.9335,0.012541,0.002569,1.0,1.0
5,2016-10-13,11.9630,0.002472,0.012541,1.0,1.0
6,2016-10-14,11.9382,-0.002073,0.002472,0.0,1.0


In [451]:
eth_data_clean.dtypes

Date           datetime64[ns]
Close_Eth             float64
ETH_Change%           float64
dtype: object

In [452]:
gold = pd.read_csv("data/Gold-Futures.csv")
gold = gold.drop(columns=["Vol.","Change %","Open","High","Low"])
gold['Date']= pd.to_datetime(gold['Date'])
gold_data = gold.dropna()
# gold_data.head()

In [453]:
gold_data = gold_data.rename(columns={'Price': 'Price_Gold'})
gold_data['Date'] = pd.to_datetime(gold_data['Date'])
gold_data['Day_Gold'] = gold_data['Date'].dt.day_name()
gold_data = gold_data.sort_values(by="Date")
gold_data = gold_data.reset_index(drop=True)
gold_data['Price_Gold'] = gold_data.Price_Gold.str.replace(',', '').astype(float)
# gold_data

In [483]:
gold_data_clean = gold_data.loc[((gold_data['Day_Gold'] != "Saturday") & (gold_data['Day_Gold'] != 'Sunday'))]
# gold_data_clean

In [488]:
# gold_data_clean = gold_data_clean.drop(columns=["Day_Gold"])
gold_data_clean["Gold_Change%"] = gold_data_clean["Price_Gold"].pct_change()
gold_data_clean['Gold_Change%_lag'] = gold_data_clean["Gold_Change%"].shift(1)

# Construction direction of GOLD movement
gold_data_clean['gold_dir'] = (gold_data_clean["Gold_Change%"]>0).astype(float)
gold_data_clean['gold_dir_lag'] = gold_data_clean['gold_dir'].shift(1)
gold_data_clean.head()

,Date,Price_Gold,Gold_Change%,Gold_Change%_lag,gold_dir,gold_dir_lag
0,2016-09-08,1400.2,NaN,NaN,0.0,NaN
1,2016-09-09,1393.1,-0.005071,NaN,0.0,0.0
2,2016-09-12,1384.4,-0.006245,-0.005071,0.0,0.0
3,2016-09-13,1380.8,-0.002600,-0.006245,0.0,0.0
4,2016-09-14,1383.3,0.001811,-0.002600,1.0,0.0


In [489]:
gold_data_clean.dtypes

Date                datetime64[ns]
Price_Gold                 float64
Gold_Change%               float64
Gold_Change%_lag           float64
gold_dir                   float64
gold_dir_lag               float64
dtype: object

In [490]:
snp = pd.read_csv("data/S&P-500.csv")
snp = snp.drop(columns=["Vol.","Change %","Open","High","Low"])
snp['Date']= pd.to_datetime(snp['Date'])
snp_data = snp.dropna()
# snp_data.head()

In [491]:
snp_data = snp_data.rename(columns={'Price': 'Price_Snp'})
snp_data['Date'] = pd.to_datetime(snp_data['Date'])
snp_data['Day_Snp'] = snp_data['Date'].dt.day_name()
snp_data = snp_data.sort_values(by="Date")
snp_data = snp_data.reset_index(drop=True)
snp_data['Price_Snp'] = snp_data.Price_Snp.str.replace(',', '').astype(float)
# snp_data

In [492]:
snp_data_clean = snp_data.loc[((snp_data['Day_Snp'] != "Saturday") & (snp_data['Day_Snp'] != 'Sunday'))]
# snp_data_clean

In [574]:
# snp_data_clean = snp_data_clean.drop(columns=["Day_Snp"])
snp_data_clean["Snp_Change%"] = snp_data_clean["Price_Snp"].pct_change()
snp_data_clean['Snp_Change%_lag'] = snp_data_clean["Snp_Change%"].shift(1)

# Construction direction of S&P movement
snp_data_clean['Snp_dir'] = (snp_data_clean["Snp_Change%"]>0).astype(float)
snp_data_clean['Snp_dir_lag'] = snp_data_clean['Snp_dir'].shift(1)
# snp_data_clean

In [494]:
usbond = pd.read_csv("data/US-10-Year-Bond.csv")
usbond = usbond.drop(columns=["Change %","Open","High","Low"])
usbond['Date']= pd.to_datetime(usbond['Date'])
usbond_data = usbond.dropna()
# usbond_data.head()

In [495]:
usbond_data = usbond_data.rename(columns={'Price': 'Price_USB'})
usbond_data['Date'] = pd.to_datetime(usbond_data['Date'])
usbond_data['Day_USB'] = usbond_data['Date'].dt.day_name()
usbond_data = usbond_data.sort_values(by="Date")
usbond_data = usbond_data.reset_index(drop=True)
# usbond_data

In [496]:
usbond_data_clean = usbond_data.loc[((usbond_data['Day_USB'] != "Saturday") & (usbond_data['Day_USB'] != 'Sunday'))]
# usbond_data_clean

In [573]:
# usbond_data_clean = usbond_data_clean.drop(columns=["Day_USB"])
usbond_data_clean["USB_Change%"] = usbond_data_clean["Price_USB"].pct_change()
usbond_data_clean['USB_Change%_lag'] = usbond_data_clean["USB_Change%"].shift(1)

# Construction direction of US BOND movement
usbond_data_clean['USB_dir'] = (usbond_data_clean["USB_Change%"]>0).astype(float)
usbond_data_clean['USB_dir_lag'] = usbond_data_clean['USB_dir'].shift(1)
# usbond_data_clean

In [531]:
usbond_data_clean.dtypes

Date               datetime64[ns]
Price_USB                 float64
USB_Change%               float64
USB_Change%_lag           float64
USB_dir                   float64
USB_dir_lag               float64
dtype: object

In [572]:
bit_1 = btc_data_clean.merge(eth_data_clean, how='left', on='Date')
# bit_1

In [570]:
bit_2 = bit_1.merge(snp_data_clean, how='left', on='Date')
# bit_2

In [571]:
bit_3 = bit_2.merge(usbond_data_clean, how='left', on='Date')
bit_3.dropna().head()

,Date,Close_Btc,BTC_Change%,BTC_Change%_lag,BTC_dir,ETH_dir_lag_x,Close_Eth,ETH_Change%,ETH_Change%_lag,ETH_dir,...,Price_Snp,Snp_Change%,Snp_Change%_lag,Snp_dir,Snp_dir_lag,Price_USB,USB_Change%,USB_Change%_lag,USB_dir,USB_dir_lag
2,2016-10-12,636.192017,-0.007612,0.035668,0.0,1.0,11.9335,0.012541,0.002569,1.0,...,2139.18,0.001147,-0.012447,1.0,0.0,1.771,0.002264,0.001133,1.0,1.0
3,2016-10-13,636.786011,0.000934,-0.007612,1.0,0.0,11.9630,0.002472,0.012541,1.0,...,2132.55,-0.003099,0.001147,0.0,1.0,1.746,-0.014116,0.002264,0.0,1.0
4,2016-10-14,640.377991,0.005641,0.000934,1.0,1.0,11.9382,-0.002073,0.002472,0.0,...,2132.98,0.000202,-0.003099,1.0,0.0,1.805,0.033792,-0.014116,1.0,0.0
5,2016-10-17,639.192993,-0.001850,0.005641,0.0,1.0,12.0050,0.005595,-0.002073,1.0,...,2126.50,-0.003038,0.000202,0.0,1.0,1.768,-0.020499,0.033792,0.0,1.0
6,2016-10-18,637.960022,-0.001929,-0.001850,0.0,0.0,12.5885,0.048605,0.005595,1.0,...,2139.60,0.006160,-0.003038,1.0,0.0,1.740,-0.015837,-0.020499,0.0,0.0


In [567]:
# bit_3.dtypes

In [566]:
gold_1 = gold_data_clean.merge(usbond_data_clean, how='left', on='Date')
# gold_1

In [565]:
gold_2 = gold_1.merge(snp_data_clean, how='left', on='Date')
# gold_2

In [542]:
gold_3 = gold_2.merge(eth_data_clean, how='left', on='Date')
gold_3 = gold_3.dropna()
gold_3 = gold_3.reset_index(drop=True)

gold_3 = gold_3[['Date', 'gold_dir', 'gold_dir_lag', 'Gold_Change%_lag', 'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
gold_3.tail()

,Date,gold_dir,gold_dir_lag,Gold_Change%_lag,USB_Change%_lag,Snp_Change%_lag,ETH_Change%_lag
1245,2021-09-30,1.0,0.0,-0.008403,-0.014230,0.001569,0.016331
1246,2021-10-01,1.0,1.0,0.019792,-0.020997,-0.011910,0.052060
1247,2021-10-04,1.0,1.0,0.000797,-0.018097,0.011494,0.101889
1248,2021-10-05,0.0,1.0,0.005232,0.010922,-0.012988,0.021942
1249,2021-10-06,1.0,0.0,-0.003790,0.031735,0.010524,0.040954


In [576]:
train = gold_3.iloc[:-100, :]
test = gold_3.iloc[-100:, :]
train.tail(), test.head()

x_train = train[['gold_dir_lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_train = train['gold_dir']
x_test = test[['gold_dir_lag', 'Gold_Change%_lag',  'USB_Change%_lag',  'Snp_Change%_lag', 'ETH_Change%_lag']]
y_test = test['gold_dir']

# x_train, y_train

In [577]:
from sklearn.linear_model import LogisticRegression
classifier = LogisticRegression(max_iter=1000)
classifier

LogisticRegression(max_iter=1000)

In [578]:
classifier.fit(x_train, y_train)

LogisticRegression(max_iter=1000)

In [579]:
print(f"Training Data Score: {classifier.score(x_train, y_train)}")
print(f"Testing Data Score: {classifier.score(x_test, y_test)}")

Training Data Score: 0.5539130434782609
Testing Data Score: 0.56


In [580]:
predictions = classifier.predict(x_test)
print(f"First 10 Predictions: {predictions[:10]}")
print(f"First 10 Actual Data: {y_test[:10].tolist()}")

First 10 Predictions: [1. 0. 1. 0. 1. 1. 1. 1. 1. 1.]
First 10 Actual Data: [1.0, 1.0, 1.0, 1.0, 0.0, 1.0, 1.0, 1.0, 0.0, 1.0]


In [581]:
pd.DataFrame({"Prediction": predictions, "Actual": y_test}).reset_index(drop=True)

,Prediction,Actual
0,1.0,1.0
1,0.0,1.0
2,1.0,1.0
3,0.0,1.0
4,1.0,0.0
...,...,...
95,1.0,1.0
96,1.0,1.0
97,1.0,1.0
98,1.0,0.0


In [589]:
features = x_train.columns.tolist()
selector = RFECV(estimator=classifier, cv=5, step=1)
_ = selector.fit(x_train, y_train)

In [590]:
selected_features = sorted(zip(selector.ranking_, features))
features_rank = pd.DataFrame(selected_features, columns=['Ranking', 'Feature'])
features_rank = features_rank.set_index('Feature')
features_rank

,Ranking
Feature,
ETH_Change%_lag,1
USB_Change%_lag,1
gold_dir_lag,2
Gold_Change%_lag,3
Snp_Change%_lag,4


In [591]:
ranked_features = []
for s in selected_features:
    if s[0] < 3:
        ranked_features.append(s[1])

In [592]:
len(ranked_features)

3

In [593]:
x_train_select = x_train[ranked_features]
x_test_select = x_test[ranked_features]

In [595]:
classifier_1 = LogisticRegression(solver='newton-cg', multi_class='auto')
classifier_1.fit(x_train_select, y_train)

print(f"Training Data Score: {classifier_1.score(x_train_select, y_train)}")
print(f"Testing Data Score: {classifier_1.score(x_test_select, y_test)}")

Training Data Score: 0.5547826086956522
Testing Data Score: 0.56


# Hyperparameter Tuning

In [596]:
classifier_2 = LogisticRegression(multi_class='auto')

param_grid = {'penalty': ["l1", "l2", 'elasticnet'],
              'solver': ['newton-cg','lbfgs','saga'],
             'C': [1,3,5]}
grid = GridSearchCV(classifier_2, param_grid, n_jobs=-2)

In [598]:
_ = grid.fit(x_train_select, y_train)

/Users/jsb/opt/anaconda3/envs/PythonAdv/lib/python3.6/site-packages/sklearn/model_selection/_search.py:925: UserWarning: One or more of the test scores are non-finite: [       nan        nan 0.54782609 0.54608696 0.54608696 0.54608696
        nan        nan        nan        nan        nan 0.54086957
 0.54608696 0.54695652 0.54608696        nan        nan        nan
        nan        nan 0.54521739 0.54956522 0.54956522 0.54956522
        nan        nan        nan]
  category=UserWarning


In [600]:
penalty = grid.best_params_['penalty']
solver = grid.best_params_['solver']
C = grid.best_params_['C']

#tuned Model

model_tuned = LogisticRegression(multi_class='auto', penalty=penalty, solver=solver, C=C)
model_tuned.fit(x_train_select, y_train)

classifier_2_training_score = round(model_tuned.score(x_train_select, y_train)*100,3)
classifier_2_test_score = round(model_tuned.score(x_test_select, y_test)*100,3)

print(f"Training Data Score: {classifier_2_training_score} %")
print(f"Testting Data Score: {classifier_2_test_score} %")

Training Data Score: 55.304 %
Testting Data Score: 54.0 %


# Randon Forest

In [653]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf = rf.fit(x_train_select, y_train)
rf.score(x_test_select, y_test)

0.58

In [654]:
sorted(zip(rf.feature_importances_, ranked_features), reverse=True)

[(0.4996445626308106, 'ETH_Change%_lag'),
 (0.489492196917064, 'USB_Change%_lag'),
 (0.010863240452125465, 'gold_dir_lag')]

In [655]:
print("n\tscore")
print("-"*20)
for n in [5,10,50,100,200,400,500,700,1000]:
    rf = RandomForestClassifier(n_estimators=n, random_state=42)
    rf = rf.fit(x_train_select, y_train)
    print(n,"\t",rf.score(x_test_select, y_test))

n	score
--------------------
5 	 0.49
10 	 0.51
50 	 0.53
100 	 0.56
200 	 0.55
400 	 0.57
500 	 0.57
700 	 0.57
1000 	 0.58


In [474]:
gold_3.dtypes

Date                datetime64[ns]
gold_dir                   float64
gold_dir_lag               float64
Gold_Change%_lag           float64
USB_Change%                float64
Snp_Change%                float64
ETH_Change%                float64
dtype: object